In [2]:
import jax
import jax.numpy as jnp
import numpy as np
from jax import random

key = random.PRNGKey(80)
random.uniform(key)

DeviceArray(0.750391, dtype=float32)

In [32]:
np.random.seed(1)
N = 4
beta = 1

sigma_z = jnp.array([[1,0],[0,-1]])
sigma_x = jnp.array([[0,1],[1,0]])
sigma_y = jnp.array([[0,1],[1,0]])

# Generate x 

spin_choices = jnp.hstack(([[0],[1]],)*N)
spin = jnp.array([0,1])
spin_up = jnp.array([[1],[0]])
spin_down= jnp.array([[0],[1]])

epsilons = np.random.rand(2,N)
eps = jnp.array(0.5 * np.random.rand(N))
p_down = 1/(1+jnp.exp(-beta * eps))
p_up = jnp.exp(-beta * eps)/(1+jnp.exp(-beta * eps))
probs = jnp.vstack((p_down,p_up))


def choose_spin(prob):
    choice = random.choice(key,spin,p=prob)
    up_func = lambda x: spin_up
    down_func = lambda x: spin_down
    if choice == 0:
        return spin_up
    else:
        return spin_down
    return jax.lax.cond(choice,up_func,down_func,0)




rand_spin =  jax.vmap(choose_spin)(probs.T)
rand_spin = rand_spin.T



ConcretizationTypeError: Abstract tracer value encountered where concrete value is expected: Traced<ShapedArray(bool[])>with<BatchTrace(level=1/0)> with
  val = DeviceArray([ True,  True,  True,  True], dtype=bool)
  batch_dim = 0
The problem arose with the `bool` function. 

See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.ConcretizationTypeError

In [36]:
[choose_spin(probs[:,i]) for i in range(N)]

[DeviceArray([[1],
              [0]], dtype=int32),
 DeviceArray([[1],
              [0]], dtype=int32),
 DeviceArray([[1],
              [0]], dtype=int32),
 DeviceArray([[1],
              [0]], dtype=int32)]

In [35]:
vec = np.random.rand(2,N)
spins = vec/ np.vstack((np.linalg.norm(vec,axis=0),)*2)
porb_up = spins[0,:] 
dist = np.prod()

In [25]:
def bloch_to_rho(vec):
    return (jnp.identity(2) + vec[0] * sigma_x + vec[1]  * sigma_y)/2

def bloch_to_rho_prod(blochVecs):

    rho_prod = bloch_to_rho(blochVecs[0,:])
    for i in range(1,blochVecs.shape[-1]):
        rho_prod = jnp.kron(rho_prod,bloch_to_rho(blochVecs[:,i]))
    return rho_prod

bloch_to_rho_prod(np.random.rand(2,10)).shape

(1024, 1024)

In [27]:

Pr = (jnp.identity(2) - sigma_z)/2


rho_prod = (1/(1 + jnp.exp(- beta*eps[0]))) * jnp.expm1(- beta* eps[0] * Pr)

def rho_prod_sacn(carry,eps_r):
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    return jnp.kron(carry,y),0



for i in range(1,N):
    eps_r = eps[i]
    y = (1/(1 + jnp.exp(- beta*eps_r))) * jnp.expm1(- beta* eps_r * Pr)
    rho_prod = jnp.kron(rho_prod,y)

#jax.lax.scan(rho_prod_sacn,init,eps[1:])
y



DeviceArray([[-0.        , -0.        ],
             [-0.        , -0.00684679]], dtype=float32)

In [26]:
jnp.sum(jnp.abs(rho_prod)>0)

DeviceArray(1, dtype=int32)

In [33]:
np.outer(x,x.T) @ x

array([0.30151134, 0.30151134, 0.        , 0.        , 0.30151134,
       0.30151134, 0.30151134, 0.30151134, 0.30151134, 0.        ,
       0.        , 0.30151134, 0.        , 0.30151134, 0.30151134,
       0.        , 0.        , 0.30151134, 0.        , 0.        ])

In [ ]:
np.random( )